In [58]:
import warnings
warnings.filterwarnings('ignore')

In [59]:
import pandas as pd
import numpy as np

In [60]:
data=pd.read_csv("all_kindle_review.csv")
df=data[["reviewText","rating"]]

In [61]:
df['rating']=np.where(df['rating']<3,0,1)

In [62]:
df['rating'].value_counts()

rating
1    8000
0    4000
Name: count, dtype: int64

In [63]:
df.head()

,reviewText,rating
0,"Jace Rankin may be short, but he's nothing to ...",1
1,Great short read. I didn't want to put it dow...,1
2,I'll start by saying this is the first of four...,1
3,Aggie is Angela Lansbury who carries pocketboo...,1
4,I did not expect this type of book to be in li...,1


In [75]:
import re
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
ps=PorterStemmer()

In [110]:
def prep(message):
    clean_message=re.sub('[^a-zA-Z0-9]',' ',message)
    lower_sms=clean_message.lower()
    splited=lower_sms.split()
    new_message=[ps.stem(word) for word in splited if not word in stopwords.words('english')]
    return new_message

In [111]:
df["reviewText"]=df["reviewText"].apply(prep)

In [112]:
df.head()

,reviewText,rating
0,"[jace, rankin, may, short, noth, mess, man, ha...",1
1,"[great, short, read, want, put, read, one, sit...",1
2,"[start, say, first, four, book, expect, 34, co...",1
3,"[aggi, angela, lansburi, carri, pocketbook, in...",1
4,"[expect, type, book, librari, plea, find, pric...",1


In [113]:
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test=train_test_split(df['reviewText'],df["rating"],test_size=0.2)

In [135]:
from gensim.models import Word2Vec  

model = Word2Vec(
    sentences=x_train,          
    vector_size=100,         
    window=5,                
    min_count=1,             
    workers=4,               
    sg=1                
)

In [136]:
import numpy as np

def sentence_vector(tokens, model):
    vectors = [model.wv[word] for word in tokens if word in model.wv]
    if len(vectors) == 0:
        return np.zeros(model.vector_size)
    return np.mean(vectors, axis=0)

# Vectorize train/test reviews
x_train_vec = np.array([sentence_vector(tokens, model) for tokens in x_train])
x_test_vec = np.array([sentence_vector(tokens, model) for tokens in x_test])


In [137]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report

forest_ml = RandomForestClassifier()
forest_ml.fit(x_train_vec, y_train)

# Predict
y_pred = forest_ml.predict(x_test_vec)

# Evaluate
print("report:", classification_report(y_test, y_pred))

report:               precision    recall  f1-score   support

           0       0.72      0.60      0.66       811
           1       0.81      0.88      0.85      1589

    accuracy                           0.79      2400
   macro avg       0.77      0.74      0.75      2400
weighted avg       0.78      0.79      0.78      2400



In [108]:
googl_modle.most_similar('MAN')

[('WOMAN', 0.6663078665733337),
 ('FOR_ALL_SEASONS', 0.6564740538597107),
 ('DRIVER', 0.6287238597869873),
 ('Wieland_Schmitz', 0.6233162879943848),
 ('THE_BULLET', 0.6118887662887573),
 ('GRANDFATHER', 0.6045420169830322),
 ("'M_YOUR", 0.6012521386146545),
 ('##-YEAR-OLD', 0.600429356098175),
 ('POLICEMAN', 0.597124457359314),
 ('ON_STREET', 0.5960486531257629)]